### $\text{MTCS-203(P): Topics in Database Management Systems}$
### $\text{Summative Test}$
*By:<br>Shiva Krishna<br>I M.Tech.(CS)<br>Regd no: 22555*


In [37]:
import os
import pandas as pd
from pathlib import Path

In [38]:
# if os.path.isfile('./input/query.txt'):
#     os.system('gedit ./input/query.txt')
if os.path.isfile('./output/output.csv'):
    os.remove('./output/output.csv')

# Query

In [39]:
query = open("./input/query.txt","r")   # open the file with query
char_replace={'Natural Join':'N_J', 
              'ORDER BY':'O_B',
              ',':' '}
line = []                     # lines in the query
for l in query:                 # append all the lines form the query to the list 'line'
    for key,value in char_replace.items():
        l = l.replace(key,value)
    line.append(l.split())

print("\nQuery Processing...")


Query Processing...


# ORDER BY

In [40]:
def order_by(ln):
    o_df = pd.read_csv('./where.csv')
    if ln[1] not in o_df.columns:
        return "Wrong attribute in the ORDER BY clause."

    
    if ((len(ln)==3) and (ln[2] == 'ASC')) or (len(ln)==2):
        o_df.sort_values(by=ln[1], inplace = True, ascending=True)
        o_df.to_csv('order.csv',index=False)
    elif ((len(ln)==3) and (ln[2] == 'DESC')): 
        o_df.sort_values(by=ln[1], inplace = True, ascending=False)
        o_df.to_csv('./order.csv',index=False)
    return 'order.csv'

# WHERE

In [41]:
def where(ln):
    df = pd.read_csv('./join.csv')
    attr = ln[1]
    val = ln[3]

    if attr not in list(df.columns):
        return "Check your WHERE clause..."
    if attr in df.columns:
        condition = ln[2]
        if df.dtypes[attr] == object:
            val = ln[3]
        elif df.dtypes[attr] == int:
            val = int(ln[3])
        elif df.dtypes[attr] == float:
            val = float(ln[3])
        else:
            return "not a defined data type"

        new_df = pd.DataFrame(columns = list(df.columns),index = range(df.shape[0]))
        rows = df.shape[0]
        k,i=0,0
        if condition == '=':
            for i in range(rows):
                if df.loc[i,attr] == val:
                    new_df.loc[k] = list(df.loc[i])
                    k+=1
        elif condition == '>=':
            for i in range(rows):
                if df.loc[i,attr] >= val:
                    new_df.loc[k] = list(df.loc[i])
                    k+=1
        elif condition == '>':
            for i in range(rows):
                if df.loc[i,attr] > val:
                    new_df.loc[k] = list(df.loc[i])
                    k+=1
        elif condition == '<=':
            for i in range(rows):
                if df.loc[i,attr] <= val:
                    new_df.loc[k] = list(df.loc[i])
                    k+=1
        else:
            for i in range(rows):
                if df.loc[i,attr] < val:
                    new_df.loc[k] = list(df.loc[i])
                    k+=1        
        new_df.drop(range(k,df.shape[0]),inplace=True)
        new_df.to_csv('./where.csv',columns = list(df.columns),index=False)
        return 'where.csv'

# Cartesian Join

In [42]:
def cartesian_join(table_1,table_2):
    intersect = []

    columns_1 = []
    columns_2 = []
    for ats_1 in list(table_1.columns):
        ats_1 = ats_1.replace(' ','')
        columns_1.append(ats_1)
    for ats_2 in list(table_2.columns):
        ats_2 = ats_2.replace(' ','')
        columns_2.append(ats_2)
    table_1.columns = columns_1
    table_2.columns = columns_2

    for att in list(table_1.columns):
        if att in list(table_2.columns):
            intersect.append(att)
    
    flag = False
    all_attr = (list(line[1][1][:-3]+table_1.columns)) + (list(line[1][2][:-3]+table_2.columns))
    for i in range(len(line)):
        for j in range(len(line[i])):
            if line[i][j] in all_attr:
                flag = True
                break
            else:
                flag = False
        if flag == True:
            break    
        
    if (len(intersect)!=0) or (flag == True):   
        new_tab = pd.DataFrame(columns = all_attr ,index = range(table_1.shape[0]*table_2.shape[0]))
    else:    
        new_tab = pd.DataFrame(columns = list(table_1.columns)+list(table_2.columns),index = range(table_1.shape[0]*table_2.shape[0]))
   
    k=0
    for i in range(table_1.shape[0]):
        for j in range(table_2.shape[0]):
            new_tab.loc[k] = list(table_1.loc[i]) + list(table_2.loc[j])
            k+=1
    
    new_tab.to_csv('join.csv',index=False)
    return "join.csv"

# Natural Join

In [43]:
def natural_join(table_1,table_2):
    intersect = []

    columns_1 = []
    columns_2 = []
    for ats_1 in list(table_1.columns):
        ats_1 = ats_1.replace(' ','')
        columns_1.append(ats_1)
    for ats_2 in list(table_2.columns):
        ats_2 = ats_2.replace(' ','')
        columns_2.append(ats_2)
    table_1.columns = columns_1
    table_2.columns = columns_2
   
    for att in list(table_1.columns):
        if att in list(table_2.columns):
            intersect.append(att)

    if (len(intersect)==0):
        return cartesian_join(table_1,table_2)

    else:        
        table_2.drop(intersect,axis=1).to_csv('after_drop.csv',index=False)
        table_3 = pd.read_csv('./after_drop.csv')
        all_attr = (list(line[1][1][:-3]+table_1.columns)) + (list(line[1][3][:-3]+table_3.columns))
        
        flag = False
        for i in range(len(line)):
            for j in range(len(line[i])):
                if line[i][j] in all_attr:
                    flag = True
                    break
                else:
                    flag = False
            if flag == True:
                break 
        if flag == True:        
            new_tab = pd.DataFrame(columns = all_attr,index = range(table_1.shape[0]*table_3.shape[0]))
        else:
            new_tab = pd.DataFrame(columns = list(table_1.columns)+list(table_3.columns),index = range(table_1.shape[0]*table_3.shape[0]))

        for att in line[0][1:]:
            if att not in list(new_tab.columns)+['*']:
                return "Give the correct attributes"

        k=0
        for i in range(table_1.shape[0]):
            for j in range(table_2.shape[0]):
                if set(table_1.loc[i,intersect]) == set(table_2.loc[j,intersect]):    
                    new_tab.loc[k] = (list(table_1.loc[i]) + list(table_3.loc[j]))
                    k+=1
        new_tab.drop(range(k,table_1.shape[0]*table_3.shape[0]),axis=0,inplace=True)
        
        os.remove("after_drop.csv")
        new_tab.to_csv('join.csv',index=False)
    return "join.csv"


# FROM

In [44]:
def from_file(params_2):

    if((len(params_2)==1) and (params_2[0][-4:]=='.csv')):
        return params_2[0]
    
    if(len(params_2)>1):
        if "N_J" not in params_2:
            params_2[0] = cartesian_join(pd.read_csv('./datasets/'+params_2[0]),pd.read_csv('./datasets/'+params_2[1]))
        else:
            params_2[0] = natural_join(pd.read_csv('./datasets/'+params_2[0]),pd.read_csv('./datasets/'+params_2[2]))
        return params_2[0]

# SELECT

In [45]:
def select(params_1):
    table = ''
    if params_1[0].upper() == 'FROM':
        table = from_file(params_1[1:])
    
    if (len(line)>2) and line[2][0].upper() == 'WHERE' and os.path.isfile('./join.csv'):
        table = where(line[2])
    
    if (len(line)>3) and line[3][0] == 'O_B' and os.path.isfile('./where.csv'):     # O_B == ORDER BY
        table = order_by(line[3])
    

    if table!='.csv' and os.path.isfile(table):
        df = pd.read_csv(table)
        if (line[0][1]=="*"):
            df.to_csv('./output/output.csv',index=False)
        else:
            df.to_csv('./output/output.csv',columns=line[0][1:],index=False)
    else:
        print(table)

# DELETE

In [46]:
def delete(params_1):
    from_table = ''
    where_table = ''
    
    if params_1[0].upper() == 'FROM':
        from_table = from_file(params_1[1:])    # operation  
        if len(params_1)>2:  
            ft = pd.read_csv(from_table)
        else:
            ft = pd.read_csv("./datasets/"+from_table)
            ft.to_csv("./join.csv",columns=ft.columns,index=False)
            ft = pd.read_csv("./join.csv")
        delete_table = ft.drop(range(0,ft.shape[0]),axis=0)

    if (len(line)>2) and line[2][0].upper() == 'WHERE' and (os.path.isfile("./"+from_table) or os.path.isfile("./datasets/"+from_table)):
        where_table = where(line[2])
        wt = pd.read_csv(where_table)
        delete_table= pd.concat([wt,ft]).drop_duplicates(keep=False)

    delete_table.to_csv('./output/output.csv',index=False)

# LIMIT

In [47]:
import math

def limit(num):
    lim_out = pd.read_csv('./output/output.csv')
    if int(num) < lim_out.shape[0]:
        new_out = lim_out.iloc[:int(num)]
    else:
        new_out = lim_out
    new_out.to_csv('./output/output.csv',index=False)

# Main Function

In [48]:
if __name__ == "__main__":
    if line[0][0].upper() == 'SELECT':
        select(line[1])
    elif line[0][0].upper() == 'DELETE':
        delete(line[1])
        
    if line[-1][0].upper() == 'LIMIT':
        limit(line[-1][1])
        
    # if os.path.isfile('./join.csv'):
    #     os.remove('join.csv')
    # if os.path.isfile('./where.csv'):
    #     os.remove('where.csv')
    # if os.path.isfile('./order.csv'):
    #     os.remove('order.csv')
    if os.path.isfile('./output/output.csv'):
        print("Output")
        os.system('code ./output/output.csv')  

Output
